In [2]:
import subprocess

def run_nvidia_smi():
    try:
        # Run the nvidia-smi command
        result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Check if the command was successful
        if result.returncode == 0:
            print("nvidia-smi output:\n")
            print(result.stdout)
        else:
            print("Error running nvidia-smi:\n")
            print(result.stderr)

    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to run nvidia-smi
run_nvidia_smi()

nvidia-smi output:

Thu Nov  7 08:21:07 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:17:00.0 Off |                    0 |
|  0%   55C    P0             87W /  300W |    3717MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------------

In [2]:
import os
# Specify the GPU index you want to check
gpu_index = 3
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_index}"
# Check if CUDA is available
import torch

import pickle

import random
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    pipeline,
)
from huggingface_hub import login
from peft import LoraConfig, get_peft_model
import time
from tqdm import tqdm
import nltk
#nltk.download('punkt')
nltk.download('punkt_tab')
torch.cuda.empty_cache()

/home/smartinez1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/smartinez1/.venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
2024-11-06 23:54:20.713072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 23:54:20.892774: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightl

In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    print(f"Number of available CUDA devices: {num_devices}")
    
    for i in range(num_devices):
        device_name = torch.cuda.get_device_name(i)
        print(f"\nDevice {i}: {device_name}")
else:
    print("CUDA is not available.")

Number of available CUDA devices: 1

Device 0: NVIDIA A40


In [ ]:
MODEL_ID = "meta-llama/Llama-3.1-8B"
MODEL_NAME = MODEL_ID.split("/")[1]
TOKEN = '' 
login(token=TOKEN)

# CHECKPOINT_PATH = "Llama-3.1-8B_2" # Define if you want to start training from checkpoint

In [4]:
# Configuration for the 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [5]:
# Load model and tokenizer with quantization settings
if not CHECKPOINT_PATH:
    model_q = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map='auto',
        attn_implementation="eager"
    )
else:
    model_q = AutoModelForCausalLM.from_pretrained(
        CHECKPOINT_PATH,
        quantization_config=bnb_config,
        device_map='auto',
        attn_implementation="eager"
    )
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.26s/it]


In [6]:
# Apply LoRA for efficient fine-tuning
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", 'k_proj', 'v_proj', 'o_proj']
)
model_q = get_peft_model(model_q, peft_config)

In [ ]:
# Load your dataset
with open('corpus_v5.pkl', 'rb') as file:
    dataset = pickle.load(file) 

In [8]:
len(dataset)

2312

In [9]:
print(dataset[-100])


Avis juridique important

 
Corrigendum to Council Regulation (EC, Euratom) No 1605/2002 of 25 June 2002 on the Financial Regulation applicable to the general budget of the European Communities (OJ L 248 of 16.9.2002)  


Official Journal L 025 , 30/01/2003 P. 0043 - 0043 
 

Corrigendum to Council Regulation (EC, Euratom) No 1605/2002 of 25 June 2002 on the Financial Regulation applicable to the general budget of the European Communities(Official Journal of the European Communities L 248 of 16 September 2002)On page 21, in Article 66(1), the final sentence:for: "The same shall apply where, through serious misconduct, he/she omits to draw up a document establishing a debt of if he/she neglects to issue a recovery order or is, without justification, late in issuing it, as the issuing of a payment order may involve third-party liability of the institution.",read: "The same shall apply where, through serious misconduct, he/she fails to draw up a document establishing an amount receivable

In [10]:
dataset = [item for item in dataset if str(item) != 'nan']

In [11]:
len(dataset)

2312

In [12]:
# Fetch LLaMA's start and end tokens
end_token_id = tokenizer.eos_token_id

In [13]:
print(tokenizer.eos_token_id)

128001


In [14]:
# Define a unique padding token for fine-tuning
custom_pad_token = "<|finetune_right_pad_id|>"
tokenizer.add_special_tokens({'pad_token': custom_pad_token})
pad_token_id = tokenizer.pad_token_id

max_length = 128  # Maximum tokens per batch

# Define a preprocessing function for phrases with token accumulation
def preprocess_and_accumulate_phrases(document, max_length):
    phrases = nltk.sent_tokenize(document)
    accumulated_tokens = []
    processed_batches = []
    ignored_phrases_count = 0
    
    for phrase in phrases:
        if len(phrase) < 50:  # Skip short phrases
            ignored_phrases_count += 1
            continue
        
        # Tokenize the phrase and add the end token
        encoding = tokenizer(phrase, return_tensors='pt', truncation=True, max_length=max_length - 1)
        input_ids = encoding['input_ids'].squeeze().tolist() + [end_token_id]
        
        # Check if adding this phrase would exceed the max token length
        if len(accumulated_tokens) + len(input_ids) > max_length:
            # If it does, save the accumulated batch and start a new one
            accumulated_tokens += [pad_token_id] * (max_length - len(accumulated_tokens))  # Pad to max_length
            processed_batches.append({
                'input_ids': torch.tensor(accumulated_tokens),  
                'labels': torch.tensor(accumulated_tokens)    
            })
            accumulated_tokens = input_ids  # Start new batch with current phrase
        else:
            accumulated_tokens.extend(input_ids)  # Append current phrase
    
    # Add the last accumulated batch if it has tokens
    if accumulated_tokens:
        accumulated_tokens += [pad_token_id] * (max_length - len(accumulated_tokens))
        processed_batches.append({
            'input_ids': torch.tensor(accumulated_tokens), 
            'labels': torch.tensor(accumulated_tokens)    
        })

    return processed_batches, ignored_phrases_count

In [15]:
# Define the file path for the processed dataset
output_file_path = "processed_filtered_dataset.pkl"

# Check if the processed file already exists
if os.path.exists(output_file_path):
    print(f"Loading processed dataset from {output_file_path}...")
    with open(output_file_path, 'rb') as f:
        processed_dataset = pickle.load(f)
    print(f"Loaded processed dataset with {len(processed_dataset)} batches.")
else:
    # Start timer for processing
    start_time = time.time()

    # Initialize variables for the processing loop
    processed_dataset = []
    total_ignored_phrases = 0

    # Process each document in the dataset
    for doc in tqdm(dataset, desc="Processing Dataset"):
        try:
            batches, ignored_count = preprocess_and_accumulate_phrases(doc, max_length)
        except TypeError as e:
            print(f"document {doc}")
            print(e)
        processed_dataset.extend(batches)
        total_ignored_phrases += ignored_count

    # Calculate processing time and output statistics
    total_time = time.time() - start_time
    print(f"Total processing time: {total_time:.2f} seconds")
    print(f"Total processed batches: {len(processed_dataset)}")
    print(f"Total ignored phrases: {total_ignored_phrases}")

    # Save the processed dataset to a pickle file
    with open(output_file_path, 'wb') as f:
        pickle.dump(processed_dataset, f)
    print(f"Filtered dataset saved to {output_file_path}")

Loading processed dataset from processed_filtered_dataset.pkl...
Loaded processed dataset with 242316 batches.


In [17]:
# Define the dataset class for training
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data  # Keep data as is

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return the item directly without moving to CPU
        return self.data[idx]


In [18]:
# Split into training and validation datasets
random.seed(42)
random.shuffle(processed_dataset)
train_examples = round(len(processed_dataset) * 0.95)

# Pass the device to the CustomDataset constructor
train_dataset = CustomDataset(processed_dataset[:train_examples])
val_dataset = CustomDataset(processed_dataset[train_examples:])

training_epochs = 3
model_dir = f"{MODEL_NAME}_{training_epochs}"

# Set up the trainer with causal language modeling and modified arguments
trainer = Trainer(
    model=model_q,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=TrainingArguments(
        per_device_train_batch_size=28,
        per_device_eval_batch_size=20,
        eval_strategy='steps',
        eval_steps=500,
        gradient_accumulation_steps=4,
        optim="adamw_8bit",
        warmup_steps=10,
        num_train_epochs=training_epochs,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=50,
        output_dir=model_dir,
    )
)

In [19]:
# Train the model
start_time = time.time()
trainer.train()
total_time = time.time() - start_time
# Print the total processing time
print(f"Total processing time: {total_time:.2f} seconds")

# Save the fine-tuned model and tokenizer
model_q.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

Step,Training Loss,Validation Loss
500,2.769300,2.781334
1000,2.721100,2.711690
1500,2.700300,2.677999
2000,2.660600,2.662708


/home/smartinez1/.venv/lib/python3.10/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error (Request ID: Root=1-672ba1af-6708579754f3381c24fc1fd7;8c84b238-1904-4799-a5f9-06c9b93ec965)

403 Forbidden: Please enable access to public gated repositories in your fine-grained token settings to view this repository..
Cannot access content at: https://huggingface.co/meta-llama/Llama-3.1-8B/resolve/main/config.json.
Make sure your token has the correct permissions. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.1-8B.
  warnings.warn(
/home/smartinez1/.venv/lib/python3.10/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in meta-llama/Llama-3.1-8B - will assume that the vocabulary was not modified.
  warnings.warn(


Total processing time: 16104.45 seconds


('Llama-3.1-8B_1/tokenizer_config.json',
 'Llama-3.1-8B_1/special_tokens_map.json',
 'Llama-3.1-8B_1/tokenizer.json')